In [1]:
import numpy as np

In [2]:
# [0:5] regular with period about 60, 
# [5:14] constant acceleration of about 12, 
# [14:20] constant deceleration of about 8,
# [20:24] regular with period about 30

distances = np.array([0, 60.3, 121.1, 180.9, 241.9, 301.0, 307.3, 331.3, 366.9, 417.0, 478.7, 550.8, 632.6, 726.5, 831.6, 928.7, 1018.0, 1099.1, 1172.3, 1237.5, 1294.7, 1325.0, 1354.6, 1384.5, 1414.9])

In [3]:
n = len(distances)

velocities = np.array([y - x for (x, y) in zip(distances, distances[1:])])
accelerations = np.array([y - x for (x, y) in zip(velocities, velocities[1:])])

In [24]:
def r(j, k):
    m = np.median(velocities[j:k])
    return np.linalg.norm(velocities[j:k] - m, ord=1)

def a(j, k):
    m = np.median(accelerations[j:k-1])
    return np.linalg.norm(accelerations[j:k-1] - m, ord=1)

In [28]:
λ = 10 # hyperparameter

D = np.zeros(n)
partitions = [[0], [0, 1]]

for i in range(2, n):
    min_k = 0
    min_cost = min(r(0, i), a(0, i)) + λ
    for k in range(2, i - 1):
        cost = D[k] + min(r(k, i), a(k, i)) + λ
        if cost < min_cost:
            min_cost, min_k = cost, k
    D[i] = min_cost
    partitions.append(partitions[min_k] + [i])

partitions[-1]

[0, 5, 13, 20, 24]

In [29]:
# import matplotlib.pyplot as plt

# %matplotlib inline

# val = 0. # y-axis
# plt.plot(timestamps, np.zeros_like(timestamps) + val, 'x')

# plt.axvline(x=timestamps[partitions[n-1][0].end_index])
# plt.axvline(x=timestamps[partitions[n-1][1].end_index])
# plt.axvline(x=timestamps[partitions[n-1][2].end_index])
# plt.axvline(x=timestamps[partitions[n-1][3].end_index])

# first_loc = 0.75*timestamps[partitions[n-1][0].start_index]+0.25*timestamps[partitions[n-1][0].end_index]
# second_loc = 0.75*timestamps[partitions[n-1][1].start_index]+0.25*timestamps[partitions[n-1][1].end_index]
# third_loc = 0.75*timestamps[partitions[n-1][2].start_index]+0.25*timestamps[partitions[n-1][2].end_index]
# fourth_loc = 0.75*timestamps[partitions[n-1][3].start_index]+0.25*timestamps[partitions[n-1][3].end_index]

# first_text = partitions[n-1][0].kind + ": " + str(partitions[n-1][0].value)
# second_text = partitions[n-1][1].kind + ": " + str(partitions[n-1][1].value)
# third_text = partitions[n-1][2].kind + ": " + str(partitions[n-1][2].value)
# fourth_text = partitions[n-1][3].kind + ": " + str(partitions[n-1][3].value)

# plt.text(first_loc, 0.02, first_text, bbox=dict(facecolor='red', alpha=0.5))
# plt.text(second_loc, 0.02, second_text, bbox=dict(facecolor='red', alpha=0.5))
# plt.text(third_loc, 0.02, third_text, bbox=dict(facecolor='red', alpha=0.5))
# plt.text(fourth_loc, 0.02, fourth_text, bbox=dict(facecolor='red', alpha=0.5))